<a href="https://colab.research.google.com/github/Mostafa772/ML_MIPT_HW/blob/lab_2/Copy_of_Overfit_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab 2
### Part 2: Dealing with overfitting

Today we work with [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) (*hint: it is available in `torchvision`*).

Your goal for today:
1. Train a FC (fully-connected) network that achieves >= 0.885 test accuracy.
2. Cause considerable overfitting by modifying the network (e.g. increasing the number of network parameters and/or layers) and demonstrate in in the appropriate way (e.g. plot loss and accurasy on train and validation set w.r.t. network complexity).
3. Try to deal with overfitting (at least partially) by using regularization techniques (Dropout/Batchnorm/...) and demonstrate the results.

__Please, write a small report describing your ideas, tries and achieved results in the end of this file.__

*Note*: Tasks 2 and 3 are interrelated, in task 3 your goal is to make the network from task 2 less prone to overfitting. Task 1 is independent from 2 and 3.

*Note 2*: We recomment to use Google Colab or other machine with GPU acceleration.

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchsummary
from sklearn.preprocessing import OneHotEncoder
import torch.utils.data
from IPython.display import clear_output
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import torch.nn.functional as F


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
# Technical function
def mkdir(path):
    if not os.path.exists(root_path):
        os.mkdir(root_path)
        print('Directory', path, 'is created!')
    else:
        print('Directory', path, 'already exists!')
        
root_path = 'fmnist'
mkdir(root_path)

Directory fmnist is created!


In [3]:
download = True
train_transform = transforms.ToTensor()
test_transform = transforms.ToTensor()
transforms.Compose((transforms.ToTensor()))


fmnist_dataset_train = torchvision.datasets.FashionMNIST(root_path, 
                                                        train=True, 
                                                        transform=train_transform,
                                                        target_transform=None,
                                                        download=download)
fmnist_dataset_test = torchvision.datasets.FashionMNIST(root_path, 
                                                       train=False, 
                                                       transform=test_transform,
                                                       target_transform=None,
                                                       download=download)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting fmnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to fmnist/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting fmnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to fmnist/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting fmnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to fmnist/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting fmnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to fmnist/FashionMNIST/raw



In [4]:
train_loader = torch.utils.data.DataLoader(fmnist_dataset_train, 
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=2)
test_loader = torch.utils.data.DataLoader(fmnist_dataset_test,
                                          batch_size=256,
                                          shuffle=False,
                                          num_workers=2)

In [5]:
len(fmnist_dataset_test)
len(fmnist_dataset_train)

60000

In [6]:
for img, label in train_loader:
    print(img.shape)
    # print(img)
    print(label.shape)
    print(label.size(0))
    break

torch.Size([128, 1, 28, 28])
torch.Size([128])
128


### Task 1
Train a network that achieves $\geq 0.885$ test accuracy. It's fine to use only Linear (`nn.Linear`) layers and activations/dropout/batchnorm. Convolutional layers might be a great use, but we will meet them a bit later.

In [7]:
class TinyNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            # Your network structure comes here
            nn.BatchNorm1d(input_shape),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(input_shape, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 10)
            # nn.Linear(input_shape, 50),
            # nn.ReLU(),
            # nn.Linear(50, num_classes),
            # nn.LogSoftmax(dim=1)
        )
    # def something(self.__class__, self)
        
    def forward(self, inp):   
        # inp = inp.view(inp.size(0), -1)  
        out = self.model(inp)
        return out

In [8]:
torchsummary.summary(TinyNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
       BatchNorm1d-2                  [-1, 784]           1,568
              ReLU-3                  [-1, 784]               0
           Dropout-4                  [-1, 784]               0
            Linear-5                  [-1, 512]         401,920
       BatchNorm1d-6                  [-1, 512]           1,024
           Dropout-7                  [-1, 512]               0
              ReLU-8                  [-1, 512]               0
            Linear-9                  [-1, 256]         131,328
      BatchNorm1d-10                  [-1, 256]             512
          Dropout-11                  [-1, 256]               0
             ReLU-12                  [-1, 256]               0
           Linear-13                  [-1, 128]          32,896
      BatchNorm1d-14                  [

In [9]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
print(input.shape, target.shape)
output = loss(input, target)
output.backward()

torch.Size([3, 5]) torch.Size([3, 5])


Your experiments come here:

In [10]:
from sklearn.metrics import accuracy_score

In [ ]:
model = TinyNeuralNetwork().to(device)
opt = torch.optim.Adagrad(model.parameters(), lr = 0.1)
loss_func = nn.CrossEntropyLoss()

# Your experiments, training and validation loops here
epochs = 20
min_valid_loss = np.inf
for e in range(epochs):
    train_accuracy = []
    test_accuracy = []
    train_loss = 0.0
    for data, labels in train_loader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Clear the gradients
        opt.zero_grad()
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = loss_func(target, labels)
        # Calculate gradients
        loss.backward()
        # Update Weights
        opt.step()
        # Calculate Loss
        train_loss += loss.item()
        predict = model.forward(data)
        y_pred = predict.argmax(axis=1)
        train_accuracy.append(accuracy_score(y_pred.cpu().numpy(), labels.cpu().numpy()))
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in test_loader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = loss_func(target,labels)
        # Calculate Loss
        valid_loss += loss.item()
        y_pred = target.argmax(axis=1)
        test_accuracy.append(accuracy_score(y_pred.cpu().numpy(), labels.cpu().numpy()))
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(test_loader)}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')
    test_accuracy.append(np.mean(test_accuracy))
    train_accuracy.append(np.mean(train_accuracy))
    print(f'Epoch {e+1} has accurcy score of test data: {test_accuracy[-1]:.4f}')

Epoch 1 		 Training Loss: 0.7069219650744375 		 Validation Loss: 0.4728592254221439
Validation Loss Decreased(inf--->18.914369) 	 Saving The Model
Epoch 1 has accurcy score of test data: 0.823
Epoch 2 		 Training Loss: 0.4024940043497187 		 Validation Loss: 0.40563946068286894
Validation Loss Decreased(18.914369--->16.225578) 	 Saving The Model
Epoch 2 has accurcy score of test data: 0.848
Epoch 3 		 Training Loss: 0.33918414698607885 		 Validation Loss: 0.3659707672894001
Validation Loss Decreased(16.225578--->14.638831) 	 Saving The Model
Epoch 3 has accurcy score of test data: 0.869
Epoch 4 		 Training Loss: 0.3003008999168746 		 Validation Loss: 0.35783392265439035
Validation Loss Decreased(14.638831--->14.313357) 	 Saving The Model
Epoch 4 has accurcy score of test data: 0.867
Epoch 5 		 Training Loss: 0.2737224862170118 		 Validation Loss: 0.35336781982332466
Validation Loss Decreased(14.313357--->14.134713) 	 Saving The Model
Epoch 5 has accurcy score of test data: 0.868
Epoch 6

In [ ]:
epochs = 5

for i in range(epochs):
    train_loss = 0.0
    for data,label in train_loader:
        if torch.cuda.is_available():
            data, label = data.cuda(), label.cuda()
        opt.zero_grad()
        
        output = model(data)
        loss = loss_func(output,label)
        loss.backward()
        # accuracy.update_state(y, logits)
        opt.step()
        
        train_loss += loss.item() * data.size(0)
    print(f'Epoch: {i+1} / {epochs} \t\t\t Training Loss:{train_loss/len(train_loader)}')

In [ ]:
test_loss = 0.0
correct, total = 0, 0

for data,label in test_loader:
    if torch.cuda.is_available():
        data, label = data.cuda(), label.cuda()
    output = model(data)
    for o,l in zip(torch.argmax(output,axis = 1),label):
        if o == l:
            correct += 1
        total += 1
    loss = loss_func(output,label)
    test_loss += loss.item() * data.size(0)
print(f'Testing Loss:{test_loss/len(test_loader)}')
print(f'Correct Predictions: {correct}/{total}')

Testing Loss:81.21260840594769
Correct Predictions: 8914/10000


In [ ]:
training_loss = 0.0
correct, total = 0, 0

for data,label in train_loader:
    if torch.cuda.is_available():
        data, label = data.cuda(), label.cuda()
    output = model(data)
    for o,l in zip(torch.argmax(output,axis = 1),label):
        if o == l:
            correct += 1
        total += 1
    loss = loss_func(output,label)
    training_loss += loss.item() * data.size(0)
print(f'Testing Loss:{training_loss/len(train_loader)}')
print(f'Correct Predictions: {correct}/{total}')

Testing Loss:43.21603296763861
Correct Predictions: 52745/60000


### Task 2: Overfit it.
Build a network that will overfit to this dataset. Demonstrate the overfitting in the appropriate way (e.g. plot loss and accurasy on train and test set w.r.t. network complexity).

*Note:* you also might decrease the size of `train` dataset to enforce the overfitting and speed up the computations.

In [55]:
train_ds, val_ds = torch.utils.data.random_split(fmnist_dataset_test, [9000, 1000])
train_ds, val_ds = torch.utils.data.random_split(val_ds, [900, 100])

In [56]:
train_loader = torch.utils.data.DataLoader(train_ds, 
                                           batch_size=512,
                                           shuffle=True,
                                           num_workers=2)
test_loader = torch.utils.data.DataLoader(val_ds,
                                          batch_size=128,
                                          shuffle=False,
                                          num_workers=2)

In [57]:
class OverfittingNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            nn.Linear(input_shape, num_classes)
        )
        
    def forward(self, inp):       
        out = self.model(inp)
        return out

In [58]:
torchsummary.summary(OverfittingNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                   [-1, 10]           7,850
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.03
Estimated Total Size (MB): 0.04
----------------------------------------------------------------


In [63]:
model = OverfittingNeuralNetwork().to(device)
opt = torch.optim.Adam(model.parameters(), lr = 0.01)
# opt = torch.optim.SGD(model.parameters(), lr = 0.01)
loss_func = nn.CrossEntropyLoss()

# Your experiments, training and validation loops here


In [64]:
epochs = 100
min_valid_loss = np.inf

for e in range(epochs):
    train_loss = 0.0
    for data, labels in train_loader:
        train_accuracy = []
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Clear the gradients
        opt.zero_grad()

        # Forward Pass
        target = model(data)

        # Find the Loss
        loss = loss_func(target, labels)

        # Calculate gradients
        loss.backward()

        # Update Weights
        opt.step()
        
        # Calculate Loss
        train_loss += loss.item()

        predict = model.forward(data)
        y_pred = predict.argmax(axis=1)
        train_accuracy.append(accuracy_score(y_pred.cpu().numpy(), labels.cpu().numpy()))
     
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in test_loader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = loss_func(target,labels)
        # Calculate Loss
        valid_loss += loss.item()
 
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(test_loader)}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')

    train_accuracy.append(np.mean(train_accuracy))
    print(f'Epoch {e} has accurcy score of train data: {train_accuracy[-1]:.3f}')

Epoch 1 		 Training Loss: 2.168192505836487 		 Validation Loss: 1.5247317552566528
Validation Loss Decreased(inf--->1.524732) 	 Saving The Model
Epoch 0 has accurcy score of train data: 0.335
Epoch 2 		 Training Loss: 1.4084157347679138 		 Validation Loss: 1.1306037902832031
Validation Loss Decreased(1.524732--->1.130604) 	 Saving The Model
Epoch 1 has accurcy score of train data: 0.549
Epoch 3 		 Training Loss: 1.1176928877830505 		 Validation Loss: 1.000498652458191
Validation Loss Decreased(1.130604--->1.000499) 	 Saving The Model
Epoch 2 has accurcy score of train data: 0.665
Epoch 4 		 Training Loss: 0.9417165219783783 		 Validation Loss: 0.8000988960266113
Validation Loss Decreased(1.000499--->0.800099) 	 Saving The Model
Epoch 3 has accurcy score of train data: 0.709
Epoch 5 		 Training Loss: 0.8278154134750366 		 Validation Loss: 0.8096267580986023
Epoch 4 has accurcy score of train data: 0.680
Epoch 6 		 Training Loss: 0.7750491201877594 		 Validation Loss: 0.7257275581359863


In [65]:
training_loss = 0.0
correct, total = 0, 0

for data,label in train_loader:
    if torch.cuda.is_available():
        data, label = data.cuda(), label.cuda()
    output = model(data)
    for o,l in zip(torch.argmax(output,axis = 1),label):
        if o == l:
            correct += 1
        total += 1
    loss = loss_func(output,label)
    training_loss += loss.item() * data.size(0)
print(f'Train Loss:{training_loss/len(train_loader)}')
print(f'Correct Predictions: {correct}/{total}')

Train Loss:67.10639226436615
Correct Predictions: 878/900


In [66]:
test_loss = 0.0
correct, total = 0, 0

for data,label in test_loader:
    if torch.cuda.is_available():
        data, label = data.cuda(), label.cuda()
    output = model(data)
    for o,l in zip(torch.argmax(output,axis = 1),label):
        if o == l:
            correct += 1
        total += 1
    loss = loss_func(output,label)
    test_loss += loss.item() * data.size(0)
print(f'Testing Loss:{test_loss/len(test_loader)}')
print(f'Correct Predictions: {correct}/{total}')

Testing Loss:60.692423582077026
Correct Predictions: 79/100


### Task 3: Fix it.
Fix the overfitted network from the previous step (at least partially) by using regularization techniques (Dropout/Batchnorm/...) and demonstrate the results. 

In [ ]:
class FixedNeuralNetwork(nn.Module):
    def __init__(self, input_shape=28*28, num_classes=10, input_channels=1):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(), # This layer converts image into a vector to use Linear layers afterwards
            # Your network structure comes here
            nn.BatchNorm1d(input_shape),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(input_shape, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, inp):       
        out = self.model(inp)
        return out

In [ ]:
torchsummary.summary(FixedNeuralNetwork().to(device), (28*28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
       BatchNorm1d-2                  [-1, 784]           1,568
              ReLU-3                  [-1, 784]               0
           Dropout-4                  [-1, 784]               0
            Linear-5                  [-1, 256]         200,960
              ReLU-6                  [-1, 256]               0
            Linear-7                   [-1, 64]          16,448
              ReLU-8                   [-1, 64]               0
            Linear-9                   [-1, 10]             650
Total params: 219,626
Trainable params: 219,626
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.84
Estimated Total Size (MB): 0.87
-------------------------------------------

In [ ]:
model = FixedNeuralNetwork().to(device)
opt = torch.optim.SGD(model.parameters(), lr = 0.1)
loss_func = nn.CrossEntropyLoss()

# Your experiments, training and validation loops here

In [ ]:
epochs = 20

min_valid_loss = np.inf

for e in range(epochs):
    train_loss = 0.0
    for data, labels in train_loader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Clear the gradients
        opt.zero_grad()
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = loss_func(target, labels)
        # Calculate gradients
        loss.backward()
        # Update Weights
        opt.step()
        # Calculate Loss
        train_loss += loss.item()
     
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in test_loader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = loss_func(target,labels)
        # Calculate Loss
        valid_loss += loss.item()
 
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(test_loader)}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')

Epoch 1 		 Training Loss: 2.292820453643799 		 Validation Loss: 32.751853942871094
Validation Loss Decreased(inf--->32.751854) 	 Saving The Model
Epoch 2 		 Training Loss: 31.06748390197754 		 Validation Loss: 53.373104095458984
Epoch 3 		 Training Loss: 41.01123809814453 		 Validation Loss: 9.764559745788574
Validation Loss Decreased(32.751854--->9.764560) 	 Saving The Model
Epoch 4 		 Training Loss: 8.791702270507812 		 Validation Loss: 5.884021759033203
Validation Loss Decreased(9.764560--->5.884022) 	 Saving The Model
Epoch 5 		 Training Loss: 5.221841812133789 		 Validation Loss: 2.5849556922912598
Validation Loss Decreased(5.884022--->2.584956) 	 Saving The Model
Epoch 6 		 Training Loss: 2.429607391357422 		 Validation Loss: 2.000626564025879
Validation Loss Decreased(2.584956--->2.000627) 	 Saving The Model
Epoch 7 		 Training Loss: 1.879091739654541 		 Validation Loss: 1.800949215888977
Validation Loss Decreased(2.000627--->1.800949) 	 Saving The Model
Epoch 8 		 Training Loss

In [ ]:
training_loss = 0.0
correct, total = 0, 0

for data,label in train_loader:
    if torch.cuda.is_available():
        data, label = data.cuda(), label.cuda()
    output = model(data)

    for o,l in zip(torch.argmax(output,axis = 1),label):
        if o == l:
            correct += 1
        total += 1
    loss = loss_func(output,label)
    training_loss += loss.item() * data.size(0)
print(f'Training Loss:{training_loss/len(train_loader)}')
print(f'Correct Predictions: {correct}/{total}')

Testing Loss:485.4287624359131
Correct Predictions: 712/900


In [ ]:
test_loss = 0.0
correct, total = 0, 0

for data,label in test_loader:
    if torch.cuda.is_available():
        data, label = data.cuda(), label.cuda()
    output = model(data)
    for o,l in zip(torch.argmax(output,axis = 1),label):
        if o == l:
            correct += 1
        total += 1
    loss = loss_func(output,label)
    test_loss += loss.item() * data.size(0)
print(f'Testing Loss:{test_loss/len(test_loader)}')
print(f'Correct Predictions: {correct}/{total}')

Testing Loss:75.48236846923828
Correct Predictions: 74/100


### Conclusions:
_Write down small report with your conclusions and your ideas._

## THE FIRST PART

> We needed several layers of Linear NN with activations/dropout/batchnorm to achieve the required accuracy. First I had several errors with the number of outcoming nodes of the NN and had many errors but then learned that we need at least 10 nodes as outcome. I tried also several techniques of activasions.


## THE OVERFITTING PART



> It was tricky for me to understand when does it really overfit especially that there is no exact way of making a NN overfit otherwise it would be easy to avoid it in real applications. **The number of iterations over the training data affects the how "fit" is the NN and if we use big batches with small data it is probably going to overfit**

## AVOIDING OVERFITTING 

> Here I saw how important to have several layers and to use activations/dropout/batchnorm techniques **IT IS SO HARD TO MAKE A NN OVERFIT WITH SGD**.





